In [ ]:
!pip install datasets transformers huggingface_hub torchaudio librosa soundfile jiwer torch protobuf seaborn bitsandbytes
!pip install accelerate -U
!pip install nvidia-smi
!pip install transformers[torch]
!pip install wandb
!pip install matplotlib seaborn
!pip install evaluate

In [ ]:
import os
from datasets import load_dataset, Audio, concatenate_datasets
import os
import re
import wandb
import torch
import numpy as np
import json
from datasets import load_dataset, Audio
from transformers import (     Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor,
    Wav2Vec2ForCTC, TrainingArguments, Trainer
)
from dataclasses import dataclass
from typing import Any, Dict, List, Union
# from dotenv import load_dotenv
from transformers import EarlyStoppingCallback
from huggingface_hub import login
import bitsandbytes
# Set processors (optional)

from datasets import load_dataset, Audio
import evaluate

num_proc = 1
num_dataloaders = 1
print(f"Cpu count: {os.cpu_count()}\nNum proc: {num_proc}\nNum dataloaders: {num_dataloaders}")
HF_API_TOKEN = os.getenv("HF_TOKEN")  # Secure environment variable for Hugging Face access tokenSXwdwQiQBwpzvZdjPObRIxFxMSQWspDVFJ"
WANDB_API_KEY = "796635845bbcb935e270354f88bafb37436ad1c4"
# Load datasets
# Login to WandB and Hugging Face
wandb.login(key=WANDB_API_KEY)
login(HF_API_TOKEN)
train = load_dataset("KYAGABA/amharic_cleaned_testset_verified", split="train")
# dev = load_dataset("KYAGABA/amharic_cleaned_testset_verified", split="train")


import unicodedata
import re

def preprocess_text(batch):
  # Convert to lowercase
  batch['transcription'] = batch['transcription'].lower()
  batch['transcription'] = re.sub(r'[^\w\s\'፡።፣፤፥፦፨]', '', batch['transcription'])
  # Replace underscores with spaces
  batch['transcription'] = re.sub(r'_', ' ', batch['transcription'])
  # Remove excessive whitespace
  batch['transcription'] = ' '.join(batch['transcription'].split())
  return batch
import librosa
import numpy as np

def get_lens(batch):
  try:
    audio_len = librosa.get_duration(y=batch['audio']['array'], sr=batch['audio']['sampling_rate'])
  except:
    del batch['audio']
    batch['audio'] = None
    audio_len = 0.0
    transcript_len = len(batch['sentence'])
    batch['audio_len'] = audio_len
    batch['transcript_len'] = transcript_len
    batch['len_ratio'] = float(audio_len)/float(transcript_len)
    batch['num_feature_vecs'] = int(np.round(audio_len * 1000 / 20))
    return batch

  transcript_len = len(batch['transcription'])

  batch['audio_len'] = audio_len
  batch['transcript_len'] = transcript_len
  batch['len_ratio'] = float(audio_len)/float(transcript_len)
  batch['num_feature_vecs'] = int(np.round(audio_len * 1000 / 20)) # seconds -> milliseconds, divide by 20 millisecond feature_win_step, round up to nearest int

  return batch

def data_checks(batch):
  audio_check = (batch['audio_len']>1.0 and batch['audio_len']<30.0)
  transcript_check = (batch['transcript_len']>10)

  input_output_ratio = float(batch['num_feature_vecs']) / float(batch['transcript_len'])
  input_output_ratio_check = input_output_ratio>1.0 # CTC algorithm assumes the input is not shorter than the ouput

  return (audio_check and transcript_check and input_output_ratio_check)

train = train.map(preprocess_text, num_proc=num_proc)
# dev = dev.map(preprocess_text, num_proc=num_proc)

try:
  train = train.map(get_lens, num_proc=num_proc)
except:
  train = train.map(get_lens, num_proc=4)

# try:
#   dev = dev.map(get_lens, num_proc=num_proc)
# except:
#   dev = dev.map(get_lens, num_proc=4)

train = train.filter(data_checks, num_proc=num_proc)
# dev = dev.filter(data_checks, num_proc=num_proc)

train_mean = np.mean(train['len_ratio'])
train_std = np.std(train['len_ratio'])

# dev_mean = np.mean(dev['len_ratio'])
# dev_std = np.std(dev['len_ratio'])

num_std_devs = 2
train = train.filter(lambda batch: (abs(batch['len_ratio'] - train_mean) - (num_std_devs * train_std)) <= 0, num_proc=num_proc)
# dev = dev.filter(lambda batch: (abs(batch['len_ratio'] - dev_mean) - (num_std_devs * dev_std)) <= 0, num_proc=num_proc)

print(f"Train hours: {sum(train['audio_len'])/3600}")


import numpy as np
def select_audio_by_hours(dataset, target_hours):
    """
    Selects audio samples from the dataset until the cumulative duration reaches the target hours.
    Returns the selected dataset and the remaining dataset.
    """
    # Convert target hours to seconds
    target_duration = target_hours * 3600

    # Sort dataset by audio length (ascending order)
    dataset = dataset.sort("audio_len")
    # Accumulate audio durations until the target duration is reached
    cumulative_duration = 0
    selected_indices = []

    for idx, audio_len in enumerate(dataset['audio_len']):
        cumulative_duration += audio_len
        if cumulative_duration > target_duration:
            break
        selected_indices.append(idx)

    # Select the entries corresponding to the accumulated indices
    selected_dataset = dataset.select(selected_indices)

    # Create a list of indices for the remaining dataset (those not in selected_indices)
    remaining_indices = [i for i in range(len(dataset)) if i not in selected_indices]
    remaining_dataset = dataset.select(remaining_indices)

    return selected_dataset, remaining_dataset

train_hours = 1
dev_hours = 1
test_hours = 5

train, remaining_train = select_audio_by_hours(train, train_hours)

dev, remaining_train_after_dev = select_audio_by_hours(remaining_train, dev_hours)

test, rem = select_audio_by_hours(remaining_train_after_dev, test_hours)


print(f"Selected {sum(train['audio_len'])/3600} hours for training.")
print(f"Selected {sum(dev['audio_len'])/3600} hours for development.")
print(f"Selected {sum(test['audio_len'])/3600} hours for additional testing.")

train = train.remove_columns(['audio_len', 'transcript_len', 'len_ratio', 'num_feature_vecs'])
dev = dev.remove_columns(['audio_len', 'transcript_len', 'len_ratio', 'num_feature_vecs'])
# Define the alphabet (including the space character explicitly)
alphabet='ሀሁሂሃሄህሆለሉሊላሌልሎሏሐሑሒሓሔሕሖሗመሙሚማሜምሞሟሠሡሢሣሤሥሦሧረሩሪራሬርሮሯሰሱሲሳሴስሶሷሸሹሺሻሼሽሾሿቀቁቂቃቄቅቆቋበቡቢባቤብቦቧቨቩቪቫቬቭቮቯተቱቲታቴትቶቷቸቹቺቻቼችቾቿኀኁኂኃኄኅኆኋነኑኒናኔንኖኗኘኙኚኛኜኝኞኟአኡኢኣኤእኦከኩኪካኬክኮኳኸኹኺኻኼኽኾዃወዉዊዋዌውዎዐዑዒዓዔዕዖዘዙዚዛዜዝዞዟዠዡዢዣዤዥዦዧየዩዪያዬይዮደዱዲዳዴድዶዷጀጁጂጃጄጅጆጇገጉጊጋጌግጎጓጠጡጢጣጤጥጦጧጨጩጪጫጬጭጮጯጰጱጲጳጴጵጶጷጸጹጺጻጼጽጾጿፀፁፂፃፄፅፆፈፉፊፋፌፍፎፏፐፑፒፓፔፕፖፗ '


# Convert the string into a list of unique characters
alphabet = sorted(list(set(alphabet)))

# Create vocabulary dictionary
vocab_dict = {v: k for k, v in enumerate(alphabet)}

# Add special tokens
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

# Save the vocabulary to a JSON file
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)


model_checkpoint = "openai/whisper-small"

language = Amharic# define the language here e.g Hausa

from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_checkpoint)

# Load WhisperTokenizer

from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained(model_checkpoint, language=language, task="transcribe")
# Combine To Create A WhisperProcessor
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained(model_checkpoint, language=language, task="transcribe")

def prepare_dataset(batch):
    # load and resample audio data
    audio = batch["audio"]
    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch
try:
  train = train.map(prepare_dataset, remove_columns=train.column_names, num_proc=num_proc)
except:
  train = train.map(prepare_dataset, remove_columns=train.column_names, num_proc=4)

try:
  dev = dev.map(prepare_dataset, remove_columns=dev.column_names, num_proc=num_proc)
except:
  dev = dev.map(prepare_dataset, remove_columns=dev.column_names, num_proc=4)

from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained(model_checkpoint)

model.generation_config.language = language
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

# Define a Data Collator

import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)
import evaluate

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

import numpy as np

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    return {"eval_wer": wer, "eval_cer": cer}


import wandb
dataset ="amharic_dataset"
language = "amh"
sample_hours = 1
version = 2
batch_size = 16
grad_acc = 1
eval_batch_size = batch_size//2
epochs =100
output_dir = f"{model_checkpoint.split('/')[-1]}-{dataset}-{language}-{sample_hours}hrs-v{version}"
wandb.init(
    project="ASR East Africa",
    entity="asr-africa-research-team",
    name=output_dir,
)
from transformers import Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    predict_with_generate=True,
    generation_max_length=225,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    gradient_accumulation_steps=grad_acc,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=epochs,
    gradient_checkpointing=True,
    fp16=True,
    learning_rate=5e-5, # define your learning rate
    lr_scheduler_type='linear',
    warmup_ratio=0.05, # define your warmpup ratio
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    optim='adamw_bnb_8bit',
# use 'adamw_bnb_8bit` for memory optimization (https://github.com/huggingface/community-events/tree/main/whisper-fine-tuning-event#tips-and-tricks)
    hub_model_id=f"asr-africa/{output_dir}",
    hub_private_repo=True,
    push_to_hub=True,
    dataloader_num_workers=num_dataloaders,
)
from transformers import Seq2SeqTrainer, EarlyStoppingCallback
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train,
    eval_dataset=dev,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[
      EarlyStoppingCallback(
        early_stopping_patience=10,
        early_stopping_threshold=1e-3
      )
    ],
)
processor.save_pretrained(training_args.output_dir)
tokenizer.save_pretrained(training_args.output_dir)
tokenizer.push_to_hub(f"asr-africa/{output_dir}")
processor.push_to_hub(f"asr-africa/{output_dir}")
trainer.train()
# Final test evaluation
#final_test_results = trainer.evaluate(eval_dataset=test_set)
# processor.push_to_hub(f"asr-africa/{output_dir}")
tainer.push_to_hub(f"asr-africa/{output_dir}")


wandb.finish()

train.cleanup_cache_files()
dev.cleanup_cache_files()
test.cleanup_cache_files()
torch.cuda.empty_cache()

In [ ]:
# Check for GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print("Not connected to a GPU")
else:
    print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install datasets transformers huggingface_hub torchaudio librosa jiwer
!apt install git-lfs
!pip install accelerate -U
!pip install nvidia-smi
!pip install transformers[torch]
!pip install wandb
!pip install matplotlib seaborn
!pip install pyctcdecode
!pip install kenlm
!pip install evaluate
!sudo apt-get install git-lfs tree
import torch
import random
import pandas as pd
from datasets import load_dataset, Audio
from IPython.display import display, HTML, Audio as IPythonAudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments
import re
import json
from collections import Counter
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import wandb
import os
from typing import Any, Dict, List, Union
from dataclasses import dataclass
from huggingface_hub import HfFolder


In [ ]:
import os
import torch
import torch.nn.functional as F
import pandas as pd
from datasets import load_dataset
from transformers import Wav2Vec2Processor, Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC
from tqdm import tqdm
from pyctcdecode import build_ctcdecoder
from huggingface_hub import login, HfApi

In [ ]:
HF_API_TOKEN = os.getenv("HF_TOKEN")  # Secure environment variable for Hugging Face access tokenSXwdwQiQBwpzvZdjPObRIxFxMSQWspDVFJ"
WANDB_API_KEY = "796635845bbcb935e270354f88bafb37436ad1c4"
# Load datasets
# Login to WandB and Hugging Face
wandb.login(key=WANDB_API_KEY)
login(HF_API_TOKEN)
# Configuration

wandb: Currently logged in as: kyagabajonah (asr-africa-research-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# ########################################
# # 1) Install huggingface_hub
# ########################################
# !pip install huggingface_hub

# ########################################
# # 2) Authenticate with Hugging Face
# ########################################
# from huggingface_hub import login

# # Replace with your actual token (read/write scope).
# HF_TOKEN = os.getenv("HF_TOKEN")  # Secure environment variable for Hugging Face access tokenSXwdwQiQBwpzvZdjPObRIxFxMSQWspDVFJ"

# # This saves credentials to Git's credential store in Colab:
# login(token=HF_TOKEN, add_to_git_credential=True)

# ########################################
# # 3) Install Git LFS (just in case)
# ########################################
# !sudo apt-get update -y
# !sudo apt-get install git-lfs
# !git lfs install

# ########################################
# # 4) Git config (name/email)
# ########################################
# !git config --global user.email "kyagabajonah@gmail.com"
# !git config --global user.name "JONAHKYAGABA"

# ########################################
# # 5) Clone your HF repo
# ########################################
# repo_owner = "asr-africa"
# repo_name  = "whisper-small-amharic_dataset-amh-200hrs-v2"
# clone_url  = f"https://huggingface.co/{repo_owner}/{repo_name}.git"
# !git clone $clone_url
# %cd $repo_name

# ########################################
# # 6) Reset to older commit
# ########################################
# commit_hash = "b51fa83"
# !git reset --hard $commit_hash

# ########################################
# # 7) Force-push the rewritten history
# ########################################
# !git push --force


In [ ]:
import os
import json
import re
import torch
import librosa
import numpy as np
import pandas as pd
from datasets import load_dataset, Audio, concatenate_datasets
from transformers import Wav2Vec2BertProcessor, Wav2Vec2ForCTC, Wav2Vec2CTCTokenizer
import evaluate
from huggingface_hub import HfApi

# Initialize WER and CER metrics
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

# Model checkpoint and repository information
model_checkpoint = "asr-africa/whisper-small-amharic_dataset-amh-200hrs-v2"
repo_id = model_checkpoint

processor = WhisperProcessor.from_pretrained(model_checkpoint, tokenizer=tokenizer)
model =WhisperForConditionalGeneration.from_pretrained(model_checkpoint)
model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define Audio feature with consistent sampling rate
audio_feature = Audio(sampling_rate=16000, mono=True)

# Define data preprocessing and helper functions
def unify_and_add_source(batch, dataset_name):
    batch['sentence'] = batch.get('transcription', batch.get('sentence', ""))
    batch['dataset_source'] = dataset_name
    return batch

def preprocess_text(batch):
    batch['sentence'] = re.sub(r'[^\w\s\'፡።፣፤፥፦፨]', '', batch['sentence'].lower()).strip()
    return batch

def get_lens(batch):
    try:
        audio_len = librosa.get_duration(y=batch['audio']['array'], sr=batch['audio']['sampling_rate'])
    except:
        audio_len = 0.0
    transcript_len = len(batch['sentence'])
    batch.update({
        'audio_len': audio_len,
        'transcript_len': transcript_len,
        'len_ratio': audio_len / transcript_len if transcript_len > 0 else 0,
        'num_feature_vecs': int(np.round(audio_len * 1000 / 20))
    })
    return batch

def data_checks(batch):
    audio_check = 1.0 < batch['audio_len'] < 30.0
    transcript_check = batch['transcript_len'] > 10
    input_output_ratio_check = (batch['num_feature_vecs'] / batch['transcript_len']) > 1.0 if batch['transcript_len'] > 0 else False
    return audio_check and transcript_check and input_output_ratio_check

def select_audio_by_hours(dataset, target_duration=4500):
    cumulative_duration, selected_indices = 0.0, []
    for idx, audio_data in enumerate(dataset["audio"]):
        duration = librosa.get_duration(y=audio_data["array"], sr=audio_data["sampling_rate"])
        cumulative_duration += duration
        if cumulative_duration > target_duration:
            break
        selected_indices.append(idx)
    return dataset.select(selected_indices)

# Dataset details with split configuration
datasets_info = {
    "AMHARIC_ALFA": {"path": "KYAGABA/AMHARIC_ALFA_DATASET_new_current", "split": "train"},
    "FLEURS": {"path": "KYAGABA/fleurs_voice_luo_new", "split": "train"},
    "COMMON_VOICE": {"path": "KYAGABA/amharic_cleaned_testset_common_voice_new", "split": "test"},
    "AMHARIC_DATASET": {"path": "KYAGABA/amharic_cleaned_testset_verified", "split": "val"}
}

# Inference function
def run_inference_on_dataset(dataset, dataset_name):
    predictions, references = [], []
    for batch in dataset:
        inputs = torch.tensor([batch["input_values"]]).to(device)
        with torch.no_grad():
            logits = model(inputs).logits
        pred_ids = torch.argmax(logits, dim=-1)
        predictions.append(processor.batch_decode(pred_ids)[0])
        references.append(batch["sentence"])
    wer = wer_metric.compute(predictions=predictions, references=references)
    cer = cer_metric.compute(predictions=predictions, references=references)
    print(f"Results for {dataset_name} - WER: {wer}, CER: {cer}")
    return {"Dataset": dataset_name, "WER": wer, "CER": cer, "Predictions": predictions, "References": references}

# Process each dataset individually
processed_datasets, all_results = [], []
for dataset_name, info in datasets_info.items():
    dataset_path, split = info["path"], info["split"]
    dataset = load_dataset(dataset_path, split=split).cast_column("audio", audio_feature)
    dataset = dataset.map(lambda batch: unify_and_add_source(batch, dataset_name)).map(preprocess_text)
    dataset = dataset.map(get_lens).filter(data_checks)
    dataset = select_audio_by_hours(dataset, target_duration=4500)
    dataset = dataset.map(lambda batch: {"input_values": processor(audio=batch["audio"]["array"], sampling_rate=batch["audio"]["sampling_rate"]).input_values[0]})
    result = run_inference_on_dataset(dataset, dataset_name)
    all_results.append(result)
    processed_datasets.append(dataset)

# Combined dataset inference
combined_test_dataset = concatenate_datasets(processed_datasets)
combined_result = run_inference_on_dataset(combined_test_dataset, "Combined Test Set")
all_results.append(combined_result)

# Log results to CSV
def log_results_to_csv(results, output_path="inference_results.csv"):
    rows = [{"Dataset": res["Dataset"], "Original Transcript": ref, "Predicted Transcript": pred, "WER": res["WER"], "CER": res["CER"]}
            for res in results for pred, ref in zip(res["Predictions"], res["References"])]
    pd.DataFrame(rows).to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

log_results_to_csv(all_results)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.93k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

...

train-00005-of-00006.parquet:   0%|          | 0.00/417M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3163 [00:00<?, ? examples/s]

Map:   0%|          | 0/3163 [00:00<?, ? examples/s]

Map:   0%|          | 0/3163 [00:00<?, ? examples/s]

Map:   0%|          | 0/3163 [00:00<?, ? examples/s]

In [ ]:
import os
import json
import re
import torch
import librosa
import numpy as np
import pandas as pd
from datasets import load_dataset, Audio, concatenate_datasets
from transformers import Wav2Vec2CTCTokenizer, AutoProcessor, pipeline
from huggingface_hub import HfApi
import evaluate

# Define the alphabet for Amharic and create vocab.json
alphabet = 'ሀሁሂሃሄህሆለሉሊላሌልሎሏሐሑሒሓሔሕሖሗመሙሚማሜምሞሟሠሡሢሣሤሥሦሧረሩሪራሬርሮሯሰሱሲሳሴስሶሷሸሹሺሻሼሽሾሿቀቁቂቃቄቅቆቋበቡቢባቤብቦቧቨቩቪቫቬቭቮቯተቱቲታቴትቶቷቸቹቺቻቼችቾቿኀኁኂኃኄኅኆኋነኑኒናኔንኖኗኘኙኚኛኜኝኞኟአኡኢኣኤእኦከኩኪካኬክኮኳኸኹኺኻኼኽኾዃወዉዊዋዌውዎዐዑዒዓዔዕዖዘዙዚዛዜዝዞዟዠዡዢዣዤዥዦዧየዩዪያዬይዮደዱዲዳዴድዶዷጀጁጂጃጄጅጆጇገጉጊጋጌግጎጓጠጡጢጣጤጥጦጧጨጩጪጫጬጭጮጯጰጱጲጳጴጵጶጷጸጹጺጻጼጽጾጿፀፁፂፃፄፅፆፈፉፊፋፌፍፎፏፐፑፒፓፔፕፖፗ '
alphabet = sorted(list(set(alphabet)))
vocab_dict = {v: k for k, v in enumerate(alphabet)}
vocab_dict["|"] = len(vocab_dict)  # Word delimiter token
vocab_dict["[UNK"] = len(vocab_dict)  # Unknown token
vocab_dict["[PAD"] = len(vocab_dict)  # Padding token
vocab_file_path = "vocab.json"
with open(vocab_file_path, "w") as vocab_file:
    json.dump(vocab_dict, vocab_file)

# Upload vocab.json to Hugging Face Hub
model_checkpoint = "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-v5"
repo_id = model_checkpoint
api = HfApi()
api.upload_file(path_or_fileobj=vocab_file_path, path_in_repo="vocab.json", repo_id=repo_id, commit_message="Adding vocabulary file")

# Load model and AutoProcessor
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(model_checkpoint)
processor = AutoProcessor.from_pretrained(model_checkpoint)
asr_pipeline = pipeline("automatic-speech-recognition", model=model_checkpoint, device=0 if torch.cuda.is_available() else -1)

# Define Audio feature with consistent sampling rate
audio_feature = Audio(sampling_rate=16000, mono=True)

# Preprocessing and helper functions
def unify_and_add_source(batch, dataset_name):
    batch['sentence'] = batch.get('transcription', batch.get('sentence', ""))
    batch['dataset_source'] = dataset_name
    return batch

def preprocess_text(batch):
    batch['sentence'] = re.sub(r'[^\w\s\'፡።፣፤፥፦፨]', '', batch['sentence'].lower()).strip()
    return batch

def get_lens(batch):
    try:
        audio_len = librosa.get_duration(y=batch['audio']['array'], sr=batch['audio']['sampling_rate'])
    except:
        audio_len = 0.0
    transcript_len = len(batch['sentence'])
    batch.update({
        'audio_len': audio_len,
        'transcript_len': transcript_len,
        'len_ratio': audio_len / transcript_len if transcript_len > 0 else 0,
        'num_feature_vecs': int(np.round(audio_len * 1000 / 20))
    })
    return batch

def data_checks(batch):
    audio_check = 1.0 < batch['audio_len'] < 30.0
    transcript_check = batch['transcript_len'] > 10
    input_output_ratio_check = (batch['num_feature_vecs'] / batch['transcript_len']) > 1.0 if batch['transcript_len'] > 0 else False
    return audio_check and transcript_check and input_output_ratio_check

def select_audio_by_hours(dataset, target_duration=4500):
    cumulative_duration, selected_indices = 0.0, []
    for idx, audio_data in enumerate(dataset["audio"]):
        duration = librosa.get_duration(y=audio_data["array"], sr=audio_data["sampling_rate"])
        cumulative_duration += duration
        if cumulative_duration > target_duration:
            break
        selected_indices.append(idx)
    return dataset.select(selected_indices)

# Dataset configuration
# Dataset details with split configuration
datasets_info = {
    "AMHARIC_ALFA": {"path": "KYAGABA/AMHARIC_ALFA_DATASET_new_current", "split": "train"},
    "FLEURS": {"path": "KYAGABA/fleurs_voice_luo_new", "split": "train"},
    "COMMON_VOICE": {"path": "KYAGABA/amharic_cleaned_testset_common_voice_new", "split": "test"}
    "AMHARIC_DATASET": {"path": "KYAGABA/amharic_cleaned_testset_verified", "split": "val"}
}
# Inference function using pipeline
def run_inference_with_pipeline(dataset, dataset_name):
    predictions, references = [], []
    for batch in dataset:
        # Only pass the audio array to the pipeline
        result = asr_pipeline(batch["audio"]["array"])
        predictions.append(result["text"])
        references.append(batch["sentence"])
    wer = evaluate.load("wer").compute(predictions=predictions, references=references)
    cer = evaluate.load("cer").compute(predictions=predictions, references=references)
    print(f"Results for {dataset_name} - WER: {wer}, CER: {cer}")
    return {"Dataset": dataset_name, "WER": wer, "CER": cer, "Predictions": predictions, "References": references}

# Process each dataset individually
processed_datasets, all_results = [], []
for dataset_name, info in datasets_info.items():
    dataset_path, split = info["path"], info["split"]
    dataset = load_dataset(dataset_path, split=split).cast_column("audio", audio_feature)
    dataset = dataset.map(lambda batch: unify_and_add_source(batch, dataset_name)).map(preprocess_text)
    dataset = dataset.map(get_lens).filter(data_checks)
    dataset = select_audio_by_hours(dataset, target_duration=4500)

    # Run inference and log results
    result = run_inference_with_pipeline(dataset, dataset_name)
    all_results.append(result)
    processed_datasets.append(dataset)

# Combined dataset inference
combined_test_dataset = concatenate_datasets(processed_datasets)
combined_result = run_inference_with_pipeline(combined_test_dataset, "Combined Test Set")
all_results.append(combined_result)

# Log results to CSV
def log_results_to_csv(results, output_path="inference_results.csv"):
    rows = [{"Dataset": res["Dataset"], "Original Transcript": ref, "Predicted Transcript": pred, "WER": res["WER"], "CER": res["CER"]}
            for res in results for pred, ref in zip(res["Predictions"], res["References"])]
    pd.DataFrame(rows).to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

log_results_to_csv(all_results)

Results for AMHARIC_ALFA - WER: 0.9635099913119027, CER: 0.5136979075083524


Map:   0%|          | 0/3163 [00:00<?, ? examples/s]

Map:   0%|          | 0/3163 [00:00<?, ? examples/s]

Map:   0%|          | 0/3163 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3163 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/2553 [00:00<?, ? examples/s]

Map:   0%|          | 0/2553 [00:00<?, ? examples/s]

Map:   0%|          | 0/2553 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2553 [00:00<?, ? examples/s]

Results for AMHARIC_DAATSET_YOHANNES - WER: 0.9460349632632379, CER: 0.4538223820862252
Results for Combined Test Set - WER: 0.9566776466663887, CER: 0.48229872685379843
Results saved to inference_results.csv


In [ ]:
import os
import json
import re
import torch
import librosa
import numpy as np
import pandas as pd
from datasets import load_dataset, Audio, concatenate_datasets
from transformers import Wav2Vec2CTCTokenizer, AutoProcessor, pipeline
from huggingface_hub import HfApi
import evaluate

# Define the alphabet for Amharic and create vocab.json
alphabet = 'ሀሁሂሃሄህሆለሉሊላሌልሎሏሐሑሒሓሔሕሖሗመሙሚማሜምሞሟሠሡሢሣሤሥሦሧረሩሪራሬርሮሯሰሱሲሳሴስሶሷሸሹሺሻሼሽሾሿቀቁቂቃቄቅቆቋበቡቢባቤብቦቧቨቩቪቫቬቭቮቯተቱቲታቴትቶቷቸቹቺቻቼችቾቿኀኁኂኃኄኅኆኋነኑኒናኔንኖኗኘኙኚኛኜኝኞኟአኡኢኣኤእኦከኩኪካኬክኮኳኸኹኺኻኼኽኾዃወዉዊዋዌውዎዐዑዒዓዔዕዖዘዙዚዛዜዝዞዟዠዡዢዣዤዥዦዧየዩዪያዬይዮደዱዲዳዴድዶዷጀጁጂጃጄጅጆጇገጉጊጋጌግጎጓጠጡጢጣጤጥጦጧጨጩጪጫጬጭጮጯጰጱጲጳጴጵጶጷጸጹጺጻጼጽጾጿፀፁፂፃፄፅፆፈፉፊፋፌፍፎፏፐፑፒፓፔፕፖፗ '
alphabet = sorted(list(set(alphabet)))
vocab_dict = {v: k for k, v in enumerate(alphabet)}
vocab_dict["|"] = len(vocab_dict)  # Word delimiter token
vocab_dict["[UNK]"] = len(vocab_dict)  # Unknown token
vocab_dict["[PAD"] = len(vocab_dict)  # Padding token
vocab_file_path = "vocab.json"
with open(vocab_file_path, "w") as vocab_file:
    json.dump(vocab_dict, vocab_file)

# List of model checkpoints
model_checkpoints = [
    "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-v5",
    "asr-africa/w2v-bert-2.0-amharic-asr-dataset-1hrs-v3",
    "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-5hrs-v5",
    "asr-africa/w2v-bert-2.0-amharic-asr-dataset-5hrs-v3",
    "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-10hrs-v6",
    "asr-africa/w2v-bert-2.0-amharic-asr-dataset-10hrs-v4",
    "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-20hrs-v6",
    "asr-africa/w2v-bert-2.0-amharic-asr-dataset-20hrs-v3",
    "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-50hrs-v6",
    "asr-africa/w2v-bert-2.0-amharic-asr-dataset-50hrs-v3"


    # Add more model checkpoints here
]

# Dataset configuration
datasets_info = {
    "AMHARIC_ALFA": {"path": "KYAGABA/AMHARIC_ALFA_DATASET_new_current", "split": "train"},
    "FLEURS": {"path": "KYAGABA/amharic_cleaned_testset_fleurs_current", "split": "train"},
    "COMMON_VOICE": {"path": "KYAGABA/amharic_cleaned_testset_common_voice_new", "split": "train"},
    "AMHARIC_DAATSET_YOHANNES": {"path":"KYAGABA/amharic_cleaned_testset_verified", "split": "val"}
}

# Helper functions
def unify_and_add_source(batch, dataset_name):
    batch['sentence'] = batch.get('transcription', batch.get('sentence', ""))
    batch['dataset_source'] = dataset_name
    return batch

def preprocess_text(batch):
    batch['sentence'] = re.sub(r'[^\w\s\'፡።፣፤፥፦፨]', '', batch['sentence'].lower()).strip()
    return batch

def get_lens(batch):
    try:
        audio_len = librosa.get_duration(y=batch['audio']['array'], sr=batch['audio']['sampling_rate'])
    except:
        audio_len = 0.0
    transcript_len = len(batch['sentence'])
    batch.update({
        'audio_len': audio_len,
        'transcript_len': transcript_len,
        'len_ratio': audio_len / transcript_len if transcript_len > 0 else 0,
        'num_feature_vecs': int(np.round(audio_len * 1000 / 20))
    })
    return batch

def data_checks(batch):
    audio_check = 1.0 < batch['audio_len'] < 30.0
    transcript_check = batch['transcript_len'] > 10
    input_output_ratio_check = (batch['num_feature_vecs'] / batch['transcript_len']) > 1.0 if batch['transcript_len'] > 0 else False
    return audio_check and transcript_check and input_output_ratio_check

def select_audio_by_hours(dataset, target_duration=4500):
    cumulative_duration, selected_indices = 0.0, []
    for idx, audio_data in enumerate(dataset["audio"]):
        duration = librosa.get_duration(y=audio_data["array"], sr=audio_data["sampling_rate"])
        cumulative_duration += duration
        if cumulative_duration > target_duration:
            break
        selected_indices.append(idx)
    return dataset.select(selected_indices)

# Inference function using pipeline
def run_inference_with_pipeline(asr_pipeline, dataset, dataset_name):
    predictions, references = [], []
    for batch in dataset:
        result = asr_pipeline(batch["audio"]["array"])
        predictions.append(result["text"])
        references.append(batch["sentence"])
    wer = evaluate.load("wer").compute(predictions=predictions, references=references)
    cer = evaluate.load("cer").compute(predictions=predictions, references=references)
    print(f"Results for {dataset_name} - WER: {wer}, CER: {cer}")
    return {"Dataset": dataset_name, "WER": wer, "CER": cer, "Predictions": predictions, "References": references}

# Processing each model checkpoint
api = HfApi()
all_model_results = []
for checkpoint in model_checkpoints:
    print(f"\nEvaluating model: {checkpoint}")

    # Check if tokenizer exists for the model; if not, upload
    try:
        tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(checkpoint)
        print("Tokenizer loaded from model checkpoint.")
    except:
        print("Tokenizer not found. Creating and uploading tokenizer for model.")
        tokenizer = Wav2Vec2CTCTokenizer(vocab_file=vocab_file_path, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
        tokenizer.push_to_hub(checkpoint, commit_message="Uploading custom tokenizer and vocabulary")

    # Create ASR pipeline with the model
    asr_pipeline = pipeline("automatic-speech-recognition", model=checkpoint, device=0 if torch.cuda.is_available() else -1)

    model_results = []
    for dataset_name, info in datasets_info.items():
        dataset_path, split = info["path"], info["split"]
        dataset = load_dataset(dataset_path, split=split).cast_column("audio", Audio(sampling_rate=16000, mono=True))
        dataset = dataset.map(lambda batch: unify_and_add_source(batch, dataset_name)).map(preprocess_text)
        dataset = dataset.map(get_lens).filter(data_checks)
        dataset = select_audio_by_hours(dataset, target_duration=4500)

        # Run inference and log results
        result = run_inference_with_pipeline(asr_pipeline, dataset, dataset_name)
        model_results.append(result)

    # Store results per model
    all_model_results.append({"Model": checkpoint, "Results": model_results})

# Log all results to CSV
def log_results_to_csv(results, output_path="inference_results.csv"):
    rows = []
    for model_data in results:
        model_name = model_data["Model"]
        for dataset_result in model_data["Results"]:
            for pred, ref in zip(dataset_result["Predictions"], dataset_result["References"]):
                rows.append({
                    "Model": model_name,
                    "Dataset": dataset_result["Dataset"],
                    "Original Transcript": ref,
                    "Predicted Transcript": pred,
                    "WER": dataset_result["WER"],
                    "CER": dataset_result["CER"]
                })
    pd.DataFrame(rows).to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")
log_results_to_csv(all_model_results)


Evaluating model: asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-v5
Tokenizer loaded from model checkpoint.
Results for AMHARIC_ALFA - WER: 0.9635099913119027, CER: 0.5136979075083524
Results for FLEURS - WER: 0.9635099913119027, CER: 0.5136979075083524
Results for COMMON_VOICE - WER: 0.9578510591832278, CER: 0.4529906956136464


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Results for AMHARIC_DAATSET_YOHANNES - WER: 0.9460349632632379, CER: 0.4538223820862252

Evaluating model: asr-africa/w2v-bert-2.0-amharic-asr-dataset-1hrs-v3
Tokenizer loaded from model checkpoint.
Results for AMHARIC_ALFA - WER: 0.49365768896611645, CER: 0.18744505011429577
Results for FLEURS - WER: 0.49365768896611645, CER: 0.18744505011429577
Results for COMMON_VOICE - WER: 0.3854553395937978, CER: 0.10682321665928224


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

...

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Results for AMHARIC_DAATSET_YOHANNES - WER: 0.5424372941474538, CER: 0.21257409189928567

Evaluating model: asr-africa/w2v-bert-2.0-amharic-asr-dataset-50hrs-v3
Tokenizer loaded from model checkpoint.
Results for AMHARIC_ALFA - WER: 0.40399652476107734, CER: 0.1632495164410058
Results for FLEURS - WER: 0.40399652476107734, CER: 0.1632495164410058
Results for COMMON_VOICE - WER: 0.2976632452500546, CER: 0.08542312804607886


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Results for AMHARIC_DAATSET_YOHANNES - WER: 0.32987078794020774, CER: 0.14595470895182128
Results saved to inference_results.csv


In [ ]:
# Processing each model checkpoint
#GET FOR COMBINED TEST SET
for checkpoint in model_checkpoints:
    print(f"\nEvaluating model: {checkpoint}")
    # Check if tokenizer exists for the model; if not, upload
    try:
        tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(checkpoint)
        print("Tokenizer loaded from model checkpoint.")
    except:
        print("Tokenizer not found. Creating and uploading tokenizer for model.")
        tokenizer = Wav2Vec2CTCTokenizer(vocab_file=vocab_file_path, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
        tokenizer.push_to_hub(checkpoint, commit_message="Uploading custom tokenizer and vocabulary")

    # Create ASR pipeline with the model
    asr_pipeline = pipeline("automatic-speech-recognition", model=checkpoint, device=0 if torch.cuda.is_available() else -1)

    # Collect 1.25-hour portions from each dataset
    combined_datasets = []
    for dataset_name, info in datasets_info.items():
        dataset_path, split = info["path"], info["split"]
        dataset = load_dataset(dataset_path, split=split).cast_column("audio", Audio(sampling_rate=16000, mono=True))
        dataset = dataset.map(lambda batch: unify_and_add_source(batch, dataset_name)).map(preprocess_text)
        dataset = dataset.map(get_lens).filter(data_checks)
        selected_dataset = select_audio_by_hours(dataset, target_duration=4500)
        combined_datasets.append(selected_dataset)  # Add selected portion to combined dataset

    # Concatenate the 1.25-hour portions into a combined dataset
    combined_test_dataset = concatenate_datasets(combined_datasets)

    # Run inference on the combined dataset only
    combined_result = run_inference_with_pipeline(asr_pipeline, combined_test_dataset, "Combined Test Set")

    # Store results for the model
    all_model_results.append({"Model": checkpoint, "Results": [combined_result]})

# Log all results to CSV
def log_results_to_csv(results, output_path="inference_results.csv"):
    rows = []
    for model_data in results:
        model_name = model_data["Model"]
        for dataset_result in model_data["Results"]:
            for pred, ref in zip(dataset_result["Predictions"], dataset_result["References"]):
                rows.append({
                    "Model": model_name,
                    "Dataset": dataset_result["Dataset"],
                    "Original Transcript": ref,
                    "Predicted Transcript": pred,
                    "WER": dataset_result["WER"],
                    "CER": dataset_result["CER"]
                })
    pd.DataFrame(rows).to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

log_results_to_csv(all_model_results)



Evaluating model: asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-v5
Tokenizer loaded from model checkpoint.


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Results for Combined Test Set - WER: 0.9566776466663887, CER: 0.48229872685379843

Evaluating model: asr-africa/w2v-bert-2.0-amharic-asr-dataset-1hrs-v3
Tokenizer loaded from model checkpoint.


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

...

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Results for Combined Test Set - WER: 0.37092940833090104, CER: 0.1429136043323971

Evaluating model: asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-20hrs-v6
Tokenizer loaded from model checkpoint.


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]